In [1]:
# each TF-Record file is for a single class and will be loaded into a separate tf.data.Dataset
# these datasets will be appended to a list, and fed into tf.experimental.sample_from_datasets

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [3]:
# directories for Train and Test TF Records, 264 files in each
train_dir = '/content/drive/My Drive/lala1/Data/Train'
test_dir = '/content/drive/My Drive/lala1/Data/Test'

In [4]:
train_tfr = os.listdir(train_dir)
test_tfr = os.listdir(test_dir)

In [5]:
def read_tfrecord(serialized_example):
    feature_description = {
          'feature0': tf.io.FixedLenFeature((), tf.string),
          'feature1': tf.io.FixedLenFeature((), tf.int64),
    }
    example = tf.io.parse_single_example(serialized_example, feature_description)

    feature0 = tf.io.parse_tensor(example['feature0'], out_type = tf.float32)
    feature1 = example['feature1']

    return feature0, feature1

In [7]:
i = 0
lis = [0]*264
for d in train_tfr:
    d_path = os.path.join(train_dir, d)
    tfrecord_dataset_train = tf.data.TFRecordDataset([d_path], compression_type="GZIP")
    dataset = tfrecord_dataset_train.map(read_tfrecord)
    lis[i] = dataset
    i = i+1
train_ds = tf.data.experimental.sample_from_datasets(lis)

In [8]:
print(i) #264

264


In [9]:
i = 0
lis = [0]*264
for d in test_tfr:
    d_path = os.path.join(test_dir, d)
    tfrecord_dataset_test = tf.data.TFRecordDataset([d_path], compression_type="GZIP")
    dataset = tfrecord_dataset_test.map(read_tfrecord)
    lis[i] = dataset
    i = i+1

test_ds = tf.data.experimental.sample_from_datasets(lis)

In [10]:
print(i) #264

264


In [11]:
def prepare_for_training(ds, shuffle_buffer_size=64, batch_size=16):
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.batch(batch_size)#.repeat(2)
    ds = ds.map(setshape)
    ds = ds.prefetch(5)
    return ds

def setshape(x, y):
    x.set_shape([None, 900000,1])
    y = tf.expand_dims(y, -1)
    y.set_shape([None,1])
    return x, y

In [12]:
train_final = prepare_for_training(train_ds)
test_final = prepare_for_training(test_ds)

In [13]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=[900000, 1]))
for rate in (1, 2, 4, 8) * 2:
    model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=5, padding='causal', activation='relu', dilation_rate=rate))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.MaxPooling1D(4))

model.add(tf.keras.layers.Conv1D(filters=16, kernel_size=5))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.MaxPooling1D(8))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(264, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 900000, 32)        192       
_________________________________________________________________
dropout (Dropout)            (None, 900000, 32)        0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 225000, 32)        0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 225000, 32)        5152      
_________________________________________________________________
dropout_1 (Dropout)          (None, 225000, 32)        0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 56250, 32)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 56250, 32)         5

In [14]:
m_cp = tf.keras.callbacks.ModelCheckpoint('Model-1', monitor='val_loss', verbose=0, save_best_only=True)

In [ ]:
history = model.fit_generator(train_final, epochs=50, validation_data=test_final, callbacks=[m_cp])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
   2275/Unknown - 749s 329ms/step - loss: 5.8400 - sparse_categorical_accuracy: 0.0169WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/my_model_cp-3/assets
2275/2275 [==============================] - 1279s 562ms/step - loss: 5.8400 - sparse_categorical_accuracy: 0.0169 - val_loss: 5.4560 - val_sparse_categorical_accuracy: 0.0123
Epoch 2/50
1159/2275 [==============>...............] - ETA: 9:12 - loss: 5.3048 - sparse_categoric